<a href="https://colab.research.google.com/github/IsaacX777/MachineLearning/blob/main/bean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
import numpy as np

In [6]:
from zipfile import ZipFile
file_name = 'dataset.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()

In [16]:
df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')

In [20]:
def load_images(df):
  images = []
  labels = df['category']
  for image_path in df['image:FILE']:
    img = Image.open(image_path)
    img = img.resize((128, 128))
    img = np.array(img) / 255.0
    images.append(img)
  encoder = LabelEncoder()
  labels = encoder.fit_transform(labels)
  return np.array(images), np.array(labels)

In [27]:
images, labels = load_images(df)
val_images, val_labels = load_images(val_df)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [31]:
model = keras.Sequential([
    keras.layers.InputLayer(shape=(128, 128, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])